<a href="https://colab.research.google.com/github/UFM-Market-Trends/UFM-Market-Trends-SDK/blob/main/examples/5_Sistema_bancario_consulta_384_SIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install pyxlsb
!pip install fpdf
import datetime
!pip install git+https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git
import ufmtrends_sdk as trends
today = datetime.date.today().strftime("%Y-%m-%d")
import requests
!pip install lxml

  Using cached pyxlsb-1.0.10-py2.py3-none-any.whl (23 kB)
  Using cached fpdf-1.7.2-py2.py3-none-any.whl
  Cloning https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git to /tmp/pip-req-build-aw893m58
  Running command git clone --filter=blob:none --quiet https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git /tmp/pip-req-build-aw893m58
  Resolved https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git to commit 9b0e7400c2e6d0399ed57c2f258b51be1b52e48e
  Preparing metadata (setup.py) ... done
  Created wheel for ufmtrends-sdk: filename=ufmtrends_sdk-0.1.7-py3-none-any.whl size=13195 sha256=1f986b85306857da1df20ff3502b7df7fa490df355f68d3a55f0a4ddf251c2eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-avbv5xc5/wheels/d8/63/72/fd94fcb7285b31cd0125721e8d1340b8842b4efc3b6d212027
Successfully built ufmtrends-sdk
  Using cached lxml-4.9.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.6 MB)


In [2]:
# # https://colab.research.google.com/notebooks/snippets/gcs.ipynb
# from google.colab import auth
# auth.authenticate_user()

# project_id = 'ufm-market-trends'
# !gcloud config set project {project_id}

# Sistema bancario - Consulta 384 SIB

In [3]:
# Recurso opcional de referencia:
# https://stackoverflow.com/questions/57164629/how-to-read-an-excel-dataframe-from-a-private-github-repository-using-pandas
# https://medium.com/towards-entrepreneurship/importing-a-csv-file-from-github-in-a-jupyter-notebook-e2c28e7e74a5
# https://stackoverflow.com/questions/25669588/convert-percent-string-to-float-in-pandas-read-csv
# https://stackoverflow.com/questions/12741092/pandas-dataframe-apply-function-to-all-columns
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
# Capitalización sistema bancario

banks_capitalization_raw_link = "https://storage.googleapis.com/ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/indicadores-financieros-instituciones-bancarias/descarga-sib-consulta-384/2023-03-Indicadores-Financieros-de-Instituciones-Bancarias-SIB-Consulta_384.xls"

download = requests.get(banks_capitalization_raw_link).content
'''
Curiosamente, el archivo producido por SIB no es un .xlsx (Excel) propiamente,
sino un .xls (XML, legible por Excel). Por tanto, la función correcta es otra.

https://stackoverflow.com/questions/9623029/python-xlrd-unsupported-format-or-corrupt-file
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html
https://stackoverflow.com/questions/38486477/get-html-table-into-pandas-dataframe-not-list-of-dataframe-objects
'''
banks_capitalization_dataframe = pd.read_html(download)[1]

banks_capitalization_dataframe = banks_capitalization_dataframe.transpose()

new_header = banks_capitalization_dataframe.iloc[0]
banks_capitalization_dataframe = banks_capitalization_dataframe[1:] 
banks_capitalization_dataframe.columns = new_header
banks_capitalization_dataframe.rename(columns={'CONCEPTO': 'date'},
                                     inplace=True)

banks_capitalization_dataframe['date'] = pd.to_datetime(
                                          banks_capitalization_dataframe['date'], 
                                          format='%d/%m/%Y')

def get_month_series(start_year, months_amount):
  start_date = datetime.datetime(year=start_year, 
                                month=1, 
                                day=1)
  
  months = pd.period_range(start_date, periods=months_amount, freq='M')
  return months
  
months = get_month_series(2008, len(banks_capitalization_dataframe['date']))
banks_capitalization_dataframe.insert(loc=1, column="month", value=months, 
                                      allow_duplicates=True) 

# Todas las columnas son strings con signo de %, formato innecesario
# (Columnas NaN simplemente retornaran NaN de nuevo.)
def pct_col_to_float(df_col):
  return df_col.str.rstrip('%').astype('float') / 100.0

banks_capitalization_dataframe.iloc[:,3:] = banks_capitalization_dataframe.iloc[:,3:].apply(pct_col_to_float,
                                                                                            axis=1)

LIQUIDITY = "LIQUIDEZ"
IMMEDIATE_LIQUIDITY = "Liquidez inmediatea (LI)"
MEDIUM_LIQUIDITY = "Liquidez mediata (LM)"
SOLVENCY = "Solvencia"
EQUITY_ASSET_RATIO = "Patrimonio con relación a activos (PA)"
EQUITY_LOANS_RATIO = "Patrimonio con relación a créditos (PCC)"
EQUITY_DEPOSITS_RATIO = "Patrimonio con relación a las captaciones (PCA)"
CAPITAL_ADEQUACY_RATIO = "Adecuación de capital (AC)"
BANKING_SYSTEM_COME = "Calce de operaciones ME con relación al patrimonio computable (COME)"
PROFITABILITY = "Rentabilidad"
RETURN_ON_EQUITY = "Rendimiento sobre patrimonio (ROE)"
RETURN_ON_ASSETS = "Rendimiento sobre activos (ROA)"
EFFICIENCY_OVER_ASSETS = "Eficiencia sobre activos"
BANKS_EFFICIENCY = "Eficiencia (EF)"
ASSETS_QUALITY = "CALIDAD DE ACTIVOS"
NON_PERFORMING_LOANS = "Cartera de créditos vencida con relación a cartera de créditos bruta (CVC)"
BANKS_COVERAGE_RATIO = "Cobertura de cartera de créditos en riesgo (CCR)" 

banks_capitalization_dataframe.columns = ["date", "month", LIQUIDITY, 
                                       IMMEDIATE_LIQUIDITY, MEDIUM_LIQUIDITY, 
                                       SOLVENCY, EQUITY_ASSET_RATIO, 
                                       EQUITY_LOANS_RATIO, EQUITY_DEPOSITS_RATIO,
                                       CAPITAL_ADEQUACY_RATIO, BANKING_SYSTEM_COME,
                                       PROFITABILITY, RETURN_ON_EQUITY, RETURN_ON_ASSETS,
                                       EFFICIENCY_OVER_ASSETS, BANKS_EFFICIENCY, 
                                       ASSETS_QUALITY, NON_PERFORMING_LOANS, 
                                       BANKS_COVERAGE_RATIO]
 
banks_capitalization_dataframe.iloc[-5:]

,date,month,LIQUIDEZ,Liquidez inmediatea (LI),Liquidez mediata (LM),Solvencia,Patrimonio con relación a activos (PA),Patrimonio con relación a créditos (PCC),Patrimonio con relación a las captaciones (PCA),Adecuación de capital (AC),Calce de operaciones ME con relación al patrimonio computable (COME),Rentabilidad,Rendimiento sobre patrimonio (ROE),Rendimiento sobre activos (ROA),Eficiencia sobre activos,Eficiencia (EF),CALIDAD DE ACTIVOS,Cartera de créditos vencida con relación a cartera de créditos bruta (CVC),Cobertura de cartera de créditos en riesgo (CCR)
179,2022-11-30,2022-11,NaN,0.1752,0.4743,NaN,0.0946,0.1754,0.1229,0.1618,0.0628,NaN,0.2369,0.0220,0.0322,0.6073,NaN,0.0133,2.5381
180,2022-12-31,2022-12,NaN,0.1983,0.4764,NaN,0.0956,0.1779,0.1238,0.1649,0.0987,NaN,0.2431,0.0227,0.0319,0.5973,NaN,0.0125,2.6254
181,2023-01-31,2023-01,NaN,0.1758,0.4757,NaN,0.0975,0.1810,0.1260,0.1659,0.0822,NaN,0.2405,0.0225,0.0321,0.6005,NaN,0.0136,2.4580
182,2023-02-28,2023-02,NaN,0.1700,0.4706,NaN,0.0979,0.1807,0.1263,0.1660,0.0750,NaN,0.2307,0.0216,0.0322,0.6047,NaN,0.0136,2.4431
183,2023-03-31,2023-03,NaN,0.1792,0.4741,NaN,0.0960,0.1773,0.1235,0.1637,0.0695,NaN,0.2336,0.0220,0.0324,0.6024,NaN,0.0135,2.4801


In [4]:
banks_capitalization_csv = "Indicadores-Financieros-Instituciones-Bancarias-Consulta-384.csv"
banks_capitalization_dataframe.to_csv(banks_capitalization_csv, index=False)

In [5]:
bucket_path = 'ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/indicadores-financieros-instituciones-bancarias'
csv_file = banks_capitalization_csv

!gsutil -h "Cache-Control:public, max-age=1" cp {csv_file} gs://{bucket_path}
!gsutil cp gs://{bucket_path}/{csv_file} gs://{bucket_path}/{'archivo'}/{today+'-'+csv_file}  
!gsutil cat gs://{bucket_path}/{csv_file}

Copying file://Indicadores-Financieros-Instituciones-Bancarias-Consulta-384.csv [Content-Type=text/csv]...
/ [1 files][ 28.4 KiB/ 28.4 KiB]                                                
Operation completed over 1 objects/28.4 KiB.                                     
Copying gs://ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/indicadores-financieros-instituciones-bancarias/Indicadores-Financieros-Instituciones-Bancarias-Consulta-384.csv [Content-Type=text/csv]...
/ [1 files][ 28.4 KiB/ 28.4 KiB]                                                
Operation completed over 1 objects/28.4 KiB.                                     
date,month,LIQUIDEZ,Liquidez inmediatea (LI),Liquidez mediata (LM),Solvencia,Patrimonio con relación a activos (PA),Patrimonio con relación a créditos (PCC),Patrimonio con relación a las captaciones (PCA),Adecuación de capital (AC),Calce de operaciones ME con relación al patrimonio computable (COME),Rentabilidad,Rendimiento sobre patrimonio (